<a href="https://colab.research.google.com/github/Gcarmnonapy7/Masterpiece_prototype_scheduling_movie_theaters/blob/main/movies_cluesterization_%26crowd_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# import depedencies
import json
import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
import kagglehub
import os
path_tmdb = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")
path_movie_box = kagglehub.dataset_download("aditya126/movies-box-office-dataset-2000-2024")

Using Colab cache for faster access to the 'tmdb-movie-metadata' dataset.
Using Colab cache for faster access to the 'movies-box-office-dataset-2000-2024' dataset.


In [15]:
print(os.listdir(path_tmdb))
print(os.listdir(path_movie_box))

['tmdb_5000_movies.csv', 'tmdb_5000_credits.csv']
['enhanced_box_office_data(2000-2024)u.csv']


In [16]:
#Import datasets

tmdb = pd.read_csv(os.path.join(path_tmdb,'tmdb_5000_movies.csv'))
box = pd.read_csv(os.path.join(path_movie_box,'enhanced_box_office_data(2000-2024)u.csv'))

Predicted Crowd Week w=∑similarityi​∑(similarityi​×crowdi,w​)​

In [17]:
print(tmdb.shape)
print(box.shape)

(4803, 20)
(5000, 13)


In [18]:
box.info(
    verbose=True,
    show_counts=True
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rank                  5000 non-null   int64  
 1   Release Group         5000 non-null   object 
 2   $Worldwide            5000 non-null   float64
 3   $Domestic             5000 non-null   float64
 4   Domestic %            5000 non-null   float64
 5   $Foreign              5000 non-null   float64
 6   Foreign %             5000 non-null   float64
 7   Year                  5000 non-null   int64  
 8   Genres                4822 non-null   object 
 9   Rating                4830 non-null   object 
 10  Vote_Count            4830 non-null   float64
 11  Original_Language     4830 non-null   object 
 12  Production_Countries  4800 non-null   object 
dtypes: float64(6), int64(2), object(5)
memory usage: 507.9+ KB


In [19]:
tmdb.info(
    verbose=True,
    show_counts=True
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [20]:
tmdb['keywords']

,keywords
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
...,...
4798,"[{""id"": 5616, ""name"": ""united states\u2013mexi..."
4799,[]
4800,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
4801,[]


In [55]:
def clean_keywords(column_keywords):
    # Handle cases where it might already be a list or is null

    if column_keywords is None or (isinstance(column_keywords, float) and pd.isna(column_keywords)):
        return []

    if isinstance(column_keywords, str):
        try:
            column_keywords = json.loads(column_keywords)
        except (json.JSONDecodeError, TypeError):
            return []

    if isinstance(column_keywords, list):
        return [item['name'] if isinstance(item, dict) else str(item) for item in column_keywords]

    return []

tmdb['keywords'] = tmdb['keywords'].apply(clean_keywords)

In [59]:
# Ensure keywords are strings before grouping
# This assumes keywords were cleaned to be lists of names (strings)
exploded_keywords = tmdb.explode('keywords')
# Calculate mean revenue per keyword string
words_freq = (
    exploded_keywords
    .groupby('keywords')
    ['revenue']
    .mean()
)

# Calculate keyword strength for each movie using the median revenue of its keywords
tmdb['keyword_strength'] = tmdb['keywords'].apply(
    lambda keywords: np.median([words_freq[word] for word in keywords]) if len(keywords) > 0 else 0
)

In [60]:
tmdb['keyword_strength'].describe()

,keyword_strength
count,4.803000e+03
mean,8.838494e+07
std,7.767139e+07
min,0.000000e+00
25%,4.317629e+07
50%,7.316093e+07
75%,1.151591e+08
max,8.772448e+08


In [22]:
tmdb['overview']

,overview
0,"In the 22nd century, a paraplegic Marine is di..."
1,"Captain Barbossa, long believed to be dead, ha..."
2,A cryptic message from Bond’s past sends him o...
3,Following the death of District Attorney Harve...
4,"John Carter is a war-weary, former military ca..."
...,...
4798,El Mariachi just wants to play his guitar and ...
4799,A newlywed couple's honeymoon is upended by th...
4800,"""Signed, Sealed, Delivered"" introduces a dedic..."
4801,When ambitious New York attorney Sam is sent t...


In [23]:
columns_to_rename_box_df = {'Release Group': 'title','$Worldwide':'worldwide_gross','Rating' : 'rating','Production_Countries' : 'countries','Original_Language':'language', 'Domestic':'contry_view'}

In [24]:
box.rename(columns=columns_to_rename_box_df,inplace=True) # title and total crowd around the world

In [26]:
# === EDA ==
import matplotlib.pyplot as plt
import seaborn as sns


sns.set_style('darkgrid')

def get_top_countries(df,column_name,top_n=10):

   df[column_name] = df[column_name].str.split(',')

   df_exploded_names = df.explode(column_name)

   df_exploded_names[column_name] = df_exploded_names[column_name].str.strip()

   return df_exploded_names[column_name].value_counts().head(top_n)

get_top_countries(box,'countries')

,count
countries,
United States of America,3153
United Kingdom,616
France,454
Japan,436
China,394
Germany,362
Canada,243
South Korea,214
Hong Kong,191


In [27]:
columns_to_drop = {'homepage','status','tagline','spoken_languages'}
tmdb = tmdb.drop(columns=columns_to_drop)

In [29]:
columns_to_log = ['revenue','budget','vote_count','keywords_strength']

def transform_log1p(data,columns_to_log):
  valid_columns = [col for col in columns_to_log if col in data.columns]
  data[valid_columns] = np.log1p(data[valid_columns])
  return data

In [30]:
# Cleaning TMDB ==> FOR FEATURE SIMILARITY

import json
import numpy as np

def clean_genres(col):
  if pd.isna(col) or col == '':
    return ''

  genres = json.loads(col)

  return ' '.join([g['name'].replace(' ','_').lower() for g in genres])

tmdb['genres'] = tmdb['genres'].apply(clean_genres)
tmdb['release_year'] = pd.to_datetime(tmdb['release_date']).dt.year


In [31]:
tmdb['genres_list'] = tmdb['genres'].apply(lambda x : x.split())
genre_exploded = tmdb.explode('genres_list')
genre_counts = genre_exploded['genres_list'].value_counts()
genre_avg_revenue = (
    genre_exploded
    .groupby('genres_list')
    ['revenue'].mean())
tmdb['genre_revenue_strength'] = tmdb['genres_list'].apply(lambda genres : np.mean([genre_avg_revenue[gen] for gen in genres] ) if len(genres) > 0 else 0)

In [32]:
tmdb[['genres_list','genre_revenue_strength']]

,genres_list,genre_revenue_strength
0,"[action, adventure, fantasy, science_fiction]",1.739210e+08
1,"[adventure, fantasy, action]",1.810758e+08
2,"[action, adventure, crime]",1.386747e+08
3,"[action, crime, drama, thriller]",8.513107e+07
4,"[action, adventure, science_fiction]",1.674433e+08
...,...,...
4798,"[action, crime, thriller]",9.613602e+07
4799,"[comedy, romance]",6.564595e+07
4800,"[comedy, drama, romance, tv_movie]",4.585203e+07
4801,[],0.000000e+00


In [33]:
print(len(tmdb[tmdb['genres_list'].apply(len) == 0]))# verify columns without genre

28


In [34]:

#Statistical engineering

def statistical_features(df):
  pass

In [35]:
#Outliers features

def detect_outliers(df):
  pass

In [36]:
tmdb.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count,release_year,genres_list,genre_revenue_strength
0,237000000,action adventure fantasy science_fiction,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,Avatar,7.2,11800,2009.0,"[action, adventure, fantasy, science_fiction]",1.739210e+08
1,300000000,adventure fantasy action,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500,2007.0,"[adventure, fantasy, action]",1.810758e+08
2,245000000,action adventure crime,206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,Spectre,6.3,4466,2015.0,"[action, adventure, crime]",1.386747e+08
3,250000000,action crime drama thriller,49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,The Dark Knight Rises,7.6,9106,2012.0,"[action, crime, drama, thriller]",8.513107e+07
4,260000000,action adventure science_fiction,49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,John Carter,6.1,2124,2012.0,"[action, adventure, science_fiction]",1.674433e+08


In [37]:
df_final = tmdb.merge(box,on='title',how='inner')